# Data Cleaning of Inventory Data 

### Step 1: Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import warnings

### Step 2: Load Raw Data

In [2]:
RAW_PATH = "../Raw_data/"
CLEAN_PATH = "../Clean_data/"

In [3]:
sales = pd.read_csv(RAW_PATH + "SalesFINAL12312016.csv")

In [4]:
purchases = pd.read_csv(RAW_PATH + "PurchasesFINAL12312016.csv")

In [5]:
invoice = pd.read_csv(RAW_PATH + "InvoicePurchases12312016.csv")

In [6]:
price = pd.read_csv(RAW_PATH + "2017PurchasePricesDec.csv")

In [7]:
beg_inv = pd.read_csv(RAW_PATH + "BegInvFINAL12312016.csv")

In [8]:
end_inv = pd.read_csv(RAW_PATH + "EndInvFINAL12312016.csv")

### Step 3: Initial Inspection & Clean `SalesFINAL12312016.csv`

In [9]:
sales.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/1/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,1/2/2016,750,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/3/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,1/8/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,1/9/2016,375,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [10]:
print(f"No. of Rows: {sales.shape[0]} \nNo. of Columns: {sales.shape[1]}")

No. of Rows: 1048575 
No. of Columns: 14


In [11]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   InventoryId     1048575 non-null  object 
 1   Store           1048575 non-null  int64  
 2   Brand           1048575 non-null  int64  
 3   Description     1048575 non-null  object 
 4   Size            1048575 non-null  object 
 5   SalesQuantity   1048575 non-null  int64  
 6   SalesDollars    1048575 non-null  float64
 7   SalesPrice      1048575 non-null  float64
 8   SalesDate       1048575 non-null  object 
 9   Volume          1048575 non-null  int64  
 10  Classification  1048575 non-null  int64  
 11  ExciseTax       1048575 non-null  float64
 12  VendorNo        1048575 non-null  int64  
 13  VendorName      1048575 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 112.0+ MB


In [12]:
#check for null values
pd.isnull(sales).sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64

In [13]:
#check for duplicated values
sales.duplicated().sum()

np.int64(0)

In [14]:
# Convert Date Column
sales['SalesDate'] = pd.to_datetime(sales['SalesDate'])

In [15]:
# Trim Text Columns
sales['VendorName'] = sales['VendorName'].str.strip()
sales['Description'] = sales['Description'].str.strip()

In [16]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   InventoryId     1048575 non-null  object        
 1   Store           1048575 non-null  int64         
 2   Brand           1048575 non-null  int64         
 3   Description     1048575 non-null  object        
 4   Size            1048575 non-null  object        
 5   SalesQuantity   1048575 non-null  int64         
 6   SalesDollars    1048575 non-null  float64       
 7   SalesPrice      1048575 non-null  float64       
 8   SalesDate       1048575 non-null  datetime64[ns]
 9   Volume          1048575 non-null  int64         
 10  Classification  1048575 non-null  int64         
 11  ExciseTax       1048575 non-null  float64       
 12  VendorNo        1048575 non-null  int64         
 13  VendorName      1048575 non-null  object        
dtypes: datetime64[ns](

In [17]:
sales['Size'].unique()

array(['750mL', '375mL 2 Pk', '50mL 4 Pk', '750mL 3 Pk', '375mL 3 Pk',
       '750mL + 3/', '200mL 3 Pk', '1.5L', '5L', 'Liter', '50mL 3 Pk',
       '100mL 4 Pk', '1.75L', '3L', '375mL', '50mL', '100mL', '50mL 5 Pk',
       '187mL', '500mL', '187mL 4 Pk', '250mL 4 Pk', '750mL 2 Pk',
       '200mL', '3/100mL', '200mL 4 Pk', '4L', '5.0 Oz', '187mL 3 Pk',
       '750mL + 2/', '300mL', '720mL', '50mL 12 Pk', '330mL', '250mL',
       '22.0 Oz', '180mL', '18L', '750mL 4 Pk', '20L'], dtype=object)

In [18]:
# Create a Copy Column
sales['size_raw'] = sales['Size'].copy()

In [19]:
# Normalize Text
sales['size_raw'] = (
    sales['size_raw']
    .str.strip()
    .str.replace(' ', '')
    .str.upper()
)

In [20]:
sales['size_raw'].unique()

array(['750ML', '375ML2PK', '50ML4PK', '750ML3PK', '375ML3PK', '750ML+3/',
       '200ML3PK', '1.5L', '5L', 'LITER', '50ML3PK', '100ML4PK', '1.75L',
       '3L', '375ML', '50ML', '100ML', '50ML5PK', '187ML', '500ML',
       '187ML4PK', '250ML4PK', '750ML2PK', '200ML', '3/100ML', '200ML4PK',
       '4L', '5.0OZ', '187ML3PK', '750ML+2/', '300ML', '720ML',
       '50ML12PK', '330ML', '250ML', '22.0OZ', '180ML', '18L', '750ML4PK',
       '20L'], dtype=object)

In [21]:
# Extract Pack Quantity
sales['pack_qty'] = (
    sales['size_raw']
    .str.extract(r'(\d+)PK')
    .astype(float)
)

sales['pack_qty'] = sales['pack_qty'].fillna(1)

In [22]:
# Handle Bonus Bottles (+ 2/, + 3/)
# Example:- 750ML+3/ → 4 bottles total

bonus = sales['size_raw'].str.extract(r'\+(\d+)/')[0].astype(float).fillna(0)

In [23]:
sales['pack_qty'] = sales['pack_qty'] + bonus

In [24]:
sales[['Size', 'pack_qty']].head(100)

,Size,pack_qty
0,750mL,1.0
1,750mL,1.0
2,750mL,1.0
3,750mL,1.0
4,375mL 2 Pk,2.0
...,...,...
95,750mL,1.0
96,750mL + 3/,4.0
97,750mL + 3/,4.0
98,750mL + 3/,4.0


In [25]:
# Extract Base Size (mL, L, Oz) & Convert everything into mL

def convert_to_ml(size):
    # Handle explicit Liter
    if size == 'LITER':
        return 1000.0
    
    num = re.search(r'\d+\.?\d*', size)
    
    if num:
        value = float(num.group())
        if 'ML' in size:
            return value
        elif 'L' in size:
            return value * 1000
        elif 'OZ' in size:
            return value * 29.5735
            
    return np.nan

In [26]:
# Apply Size Conversion
sales['unit_size_ml'] = sales['size_raw'].apply(convert_to_ml)

In [27]:
sales[['size_raw', 'unit_size_ml']].head(100)

,size_raw,unit_size_ml
0,750ML,750.0
1,750ML,750.0
2,750ML,750.0
3,750ML,750.0
4,375ML2PK,375.0
...,...,...
95,750ML,750.0
96,750ML+3/,750.0
97,750ML+3/,750.0
98,750ML+3/,750.0


In [28]:
mask_frac = sales['size_raw'].str.contains(r'^\d+/\d+ML', regex=True, na=False)

In [29]:
fraction_parts = sales.loc[mask_frac, 'size_raw'].str.extract(r'(\d+)/(\d+)ML').astype(float)
sales.loc[mask_frac, 'pack_qty'] = fraction_parts[0]      
sales.loc[mask_frac, 'unit_size_ml'] = fraction_parts[1]  

In [30]:
# Calculate Total Size
sales['Size_ML'] = sales['unit_size_ml'] * sales['pack_qty']

In [31]:
# Validate Results
sales[['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']].head(100)

,Size,unit_size_ml,pack_qty,Size_ML
0,750mL,750.0,1.0,750.0
1,750mL,750.0,1.0,750.0
2,750mL,750.0,1.0,750.0
3,750mL,750.0,1.0,750.0
4,375mL 2 Pk,375.0,2.0,750.0
...,...,...,...,...
95,750mL,750.0,1.0,750.0
96,750mL + 3/,750.0,4.0,3000.0
97,750mL + 3/,750.0,4.0,3000.0
98,750mL + 3/,750.0,4.0,3000.0


In [32]:
# List of specific values
target_sizes = [
    '750mL', 
    '375mL 2 Pk', 
    '750mL + 3/', 
    'Liter', 
    '1.75L', 
    '5.0 Oz', 
    '3/100mL', 
    '20L'
]

# Filter and display
result = sales[sales['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

              Size  unit_size_ml  pack_qty     Size_ML
0            750mL      750.0000       1.0    750.0000
424204  375mL 2 Pk      375.0000       2.0    750.0000
681032  750mL + 3/      750.0000       4.0   3000.0000
29756        Liter     1000.0000       1.0   1000.0000
509578       1.75L     1750.0000       1.0   1750.0000
608108      5.0 Oz      147.8675       1.0    147.8675
501227     3/100mL      100.0000       3.0    300.0000
506194         20L    20000.0000       1.0  20000.0000


In [33]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   InventoryId     1048575 non-null  object        
 1   Store           1048575 non-null  int64         
 2   Brand           1048575 non-null  int64         
 3   Description     1048575 non-null  object        
 4   Size            1048575 non-null  object        
 5   SalesQuantity   1048575 non-null  int64         
 6   SalesDollars    1048575 non-null  float64       
 7   SalesPrice      1048575 non-null  float64       
 8   SalesDate       1048575 non-null  datetime64[ns]
 9   Volume          1048575 non-null  int64         
 10  Classification  1048575 non-null  int64         
 11  ExciseTax       1048575 non-null  float64       
 12  VendorNo        1048575 non-null  int64         
 13  VendorName      1048575 non-null  object        
 14  size_raw        10

In [34]:
sales.drop(['Size','size_raw', 'unit_size_ml', 'pack_qty'], axis=1, inplace=True)

In [35]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   InventoryId     1048575 non-null  object        
 1   Store           1048575 non-null  int64         
 2   Brand           1048575 non-null  int64         
 3   Description     1048575 non-null  object        
 4   SalesQuantity   1048575 non-null  int64         
 5   SalesDollars    1048575 non-null  float64       
 6   SalesPrice      1048575 non-null  float64       
 7   SalesDate       1048575 non-null  datetime64[ns]
 8   Volume          1048575 non-null  int64         
 9   Classification  1048575 non-null  int64         
 10  ExciseTax       1048575 non-null  float64       
 11  VendorNo        1048575 non-null  int64         
 12  VendorName      1048575 non-null  object        
 13  Size_ML         1048575 non-null  float64       
dtypes: datetime64[ns](

In [36]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName', 'Size_ML'],
      dtype='object')

In [37]:
# 1. Define the new column order
new_order = [
    'InventoryId', 'Store', 'Brand', 'Description',
    'Size_ML','SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 
    'Volume', 'Classification', 'ExciseTax', 'VendorNo', 
    'VendorName'
]

# 2. Reorder the dataframe
sales = sales[new_order]

# 3. Verify the change
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size_ML',
       'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume',
       'Classification', 'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [38]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   InventoryId     1048575 non-null  object        
 1   Store           1048575 non-null  int64         
 2   Brand           1048575 non-null  int64         
 3   Description     1048575 non-null  object        
 4   Size_ML         1048575 non-null  float64       
 5   SalesQuantity   1048575 non-null  int64         
 6   SalesDollars    1048575 non-null  float64       
 7   SalesPrice      1048575 non-null  float64       
 8   SalesDate       1048575 non-null  datetime64[ns]
 9   Volume          1048575 non-null  int64         
 10  Classification  1048575 non-null  int64         
 11  ExciseTax       1048575 non-null  float64       
 12  VendorNo        1048575 non-null  int64         
 13  VendorName      1048575 non-null  object        
dtypes: datetime64[ns](

In [39]:
sales.shape

(1048575, 14)

In [40]:
# Approx size in MB
sales.memory_usage(deep=True).sum() / (1024**2)

np.float64(293.2692279815674)

### Save Cleaned Data

In [41]:
sales.to_csv(CLEAN_PATH + "Final_Sales.csv", index=False)

### Step 4: Initial Inspection & Clean `PurchasesFINAL12312016.csv.csv`

In [42]:
purchases.head()

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.32,5,106.60,1


In [43]:
print(f"No. of Rows: {purchases.shape[0]} \nNo. of Columns: {purchases.shape[1]}")

No. of Rows: 2372474 
No. of Columns: 16


In [44]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   InventoryId     object 
 1   Store           int64  
 2   Brand           int64  
 3   Description     object 
 4   Size            object 
 5   VendorNumber    int64  
 6   VendorName      object 
 7   PONumber        int64  
 8   PODate          object 
 9   ReceivingDate   object 
 10  InvoiceDate     object 
 11  PayDate         object 
 12  PurchasePrice   float64
 13  Quantity        int64  
 14  Dollars         float64
 15  Classification  int64  
dtypes: float64(2), int64(6), object(8)
memory usage: 289.6+ MB


In [45]:
#check for null values
pd.isnull(purchases).sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              3
VendorNumber      0
VendorName        0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
dtype: int64

In [46]:
#check for duplicated values
purchases.duplicated().sum()

np.int64(0)

In [47]:
# Convert Date Columns
date_cols = ['PODate', 'ReceivingDate', 'InvoiceDate', 'PayDate']
for col in date_cols:
    purchases[col] = pd.to_datetime(purchases[col])

In [48]:
# Trim Text Columns
purchases['VendorName'] = purchases['VendorName'].str.strip()
purchases['Description'] = purchases['Description'].str.strip()

In [49]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype         
---  ------          -----         
 0   InventoryId     object        
 1   Store           int64         
 2   Brand           int64         
 3   Description     object        
 4   Size            object        
 5   VendorNumber    int64         
 6   VendorName      object        
 7   PONumber        int64         
 8   PODate          datetime64[ns]
 9   ReceivingDate   datetime64[ns]
 10  InvoiceDate     datetime64[ns]
 11  PayDate         datetime64[ns]
 12  PurchasePrice   float64       
 13  Quantity        int64         
 14  Dollars         float64       
 15  Classification  int64         
dtypes: datetime64[ns](4), float64(2), int64(6), object(4)
memory usage: 289.6+ MB


In [50]:
purchases[purchases['Size'].isnull()]

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
1109668,34_PITMERDEN_3121,34,3121,Pinnacle Rainbow Sherbet,NaN,12546,JIM BEAM BRANDS COMPANY,10938,2016-06-27,2016-07-04,2016-07-13,2016-08-16,6.93,7,48.51,1
1112426,34_PITMERDEN_5678,34,5678,Skinnygirl Pina Colada,NaN,12546,JIM BEAM BRANDS COMPANY,10938,2016-06-27,2016-07-09,2016-07-13,2016-08-16,6.93,6,41.58,1
1116302,39_EASTHALLOW_15365,39,15365,Alabaster 07 Tinta de Toro,NaN,9552,M S WALKER INC,10972,2016-06-29,2016-07-07,2016-07-13,2016-08-21,91.83,1,91.83,2


In [51]:
# Fill nulls with a specific string
purchases['Size'] = purchases['Size'].fillna('Unknown')

In [52]:
#check for null values
pd.isnull(purchases).sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
VendorNumber      0
VendorName        0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
dtype: int64

In [53]:
purchases['Size'].unique()

array(['750mL', '1.75L', '200mL', '1.5L', '50mL', 'Liter', '375mL',
       '100mL', '750mL 2 Pk', '3L', '187mL 4 Pk', '250mL 4 Pk', '500mL',
       '187mL', '100mL 4 Pk', '5L', '4L', '375mL 2 Pk', '187mL 3 Pk',
       '250mL', '180mL', '50mL 5 Pk', '200mL 4 Pk', '5.0 Oz', '18L',
       '300mL', '720mL', '50mL 3 Pk', '50mL 4 Pk', '330mL', '750mL 3 Pk',
       '3/100mL', '750mL + 2/', '750mL 4 Pk', '200mL 3 Pk', '375mL 3 Pk',
       '50mL 12 Pk', '22.0 Oz', '750mL + 3/', '200mL 5 Pk', '750mL + 1/',
       '5/2 oz', '6L', '20L', '400mL', 'Unknown', '25.0', '187mL 2 Pk',
       '19.5L', '750mL 6 Pk', '9L', '128.0 Gal'], dtype=object)

In [54]:
purchases[purchases['Size']== '25.0']

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
1189439,49_GARIGILL_169,49,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,90,1185.30,1
1189591,55_DRY GULCH_169,55,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,90,1185.30,1
1189681,41_LARNWICK_169,41,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,89,1172.13,1
1189684,32_MOUNTMEND_169,32,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,90,1185.30,1
1189842,6_GOULCREST_169,6,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,42,553.14,1
1189868,2_ASHBORNE_169,2,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,89,1172.13,1
1189883,79_BALLYMENA_169,79,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-16,2016-07-25,2016-08-22,13.17,90,1185.30,1
1189886,23_ARBINGTON_169,23,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,89,1172.13,1
1189948,1_HARDERSFIELD_169,1,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-16,2016-07-25,2016-08-22,13.17,90,1185.30,1
1189968,47_PELLA'S WISH_169,47,169,Twisted Shotz Multi Pack,25.0,9165,ULTRA BEVERAGE COMPANY LLP,11044,2016-07-03,2016-07-15,2016-07-25,2016-08-22,13.17,5,65.85,1


In [55]:
purchases['Size'] = purchases['Size'].astype(str).str.strip().replace('25.0', '25.0 Oz')

In [56]:
purchases['Size'].unique()

array(['750mL', '1.75L', '200mL', '1.5L', '50mL', 'Liter', '375mL',
       '100mL', '750mL 2 Pk', '3L', '187mL 4 Pk', '250mL 4 Pk', '500mL',
       '187mL', '100mL 4 Pk', '5L', '4L', '375mL 2 Pk', '187mL 3 Pk',
       '250mL', '180mL', '50mL 5 Pk', '200mL 4 Pk', '5.0 Oz', '18L',
       '300mL', '720mL', '50mL 3 Pk', '50mL 4 Pk', '330mL', '750mL 3 Pk',
       '3/100mL', '750mL + 2/', '750mL 4 Pk', '200mL 3 Pk', '375mL 3 Pk',
       '50mL 12 Pk', '22.0 Oz', '750mL + 3/', '200mL 5 Pk', '750mL + 1/',
       '5/2 oz', '6L', '20L', '400mL', 'Unknown', '25.0 Oz', '187mL 2 Pk',
       '19.5L', '750mL 6 Pk', '9L', '128.0 Gal'], dtype=object)

In [57]:
# Create a Copy Column
purchases['size_raw'] = purchases['Size'].copy()

In [58]:
# Normalize Text
purchases['size_raw'] = (
    purchases['size_raw']
    .str.strip()
    .str.replace(' ', '')
    .str.upper()
)

In [59]:
purchases['size_raw'].unique()

array(['750ML', '1.75L', '200ML', '1.5L', '50ML', 'LITER', '375ML',
       '100ML', '750ML2PK', '3L', '187ML4PK', '250ML4PK', '500ML',
       '187ML', '100ML4PK', '5L', '4L', '375ML2PK', '187ML3PK', '250ML',
       '180ML', '50ML5PK', '200ML4PK', '5.0OZ', '18L', '300ML', '720ML',
       '50ML3PK', '50ML4PK', '330ML', '750ML3PK', '3/100ML', '750ML+2/',
       '750ML4PK', '200ML3PK', '375ML3PK', '50ML12PK', '22.0OZ',
       '750ML+3/', '200ML5PK', '750ML+1/', '5/2OZ', '6L', '20L', '400ML',
       'UNKNOWN', '25.0OZ', '187ML2PK', '19.5L', '750ML6PK', '9L',
       '128.0GAL'], dtype=object)

In [60]:
# Extract Pack Quantity
purchases['pack_qty'] = (
    purchases['size_raw']
    .str.extract(r'(\d+)PK')
    .astype(float)
)

purchases['pack_qty'] = purchases['pack_qty'].fillna(1)

In [61]:
# Handle Bonus Bottles (+ 2/, + 3/)
# Example:- 750ML+3/ → 4 bottles total

bonus = purchases['size_raw'].str.extract(r'\+(\d+)/')[0].astype(float).fillna(0)

In [62]:
purchases['pack_qty'] = purchases['pack_qty'] + bonus

In [63]:
purchases[['Size', 'pack_qty']].head(100)

,Size,pack_qty
0,750mL,1.0
1,1.75L,1.0
2,1.75L,1.0
3,1.75L,1.0
4,750mL,1.0
...,...,...
95,1.5L,1.0
96,1.75L,1.0
97,Liter,1.0
98,750mL,1.0


In [64]:
# Extract Base Size (mL, L, Oz) & Convert everything into mL
def convert_to_ml(size):

    # Handle Unknown
    if size == 'UNKNOWN':
        return np.nan

    # Gallons
    gal = re.match(r'(\d+\.?\d*)\s*GAL', size)
    if gal:
        return float(gal.group(1)) * 3785.41
    
    # Handle explicit Liter
    if size == 'LITER':
        return 1000.0

     # Ounces
    oz = re.match(r'(\d+\.?\d*)\s*OZ', size)
    if oz:
        return float(oz.group(1)) * 29.5735
    
    num = re.search(r'\d+\.?\d*', size)
    
    if num:
        value = float(num.group())
        if 'ML' in size:
            return value
        elif 'L' in size:
            return value * 1000
            
    return np.nan

In [65]:
# Apply Size Conversion
purchases['unit_size_ml'] = purchases['size_raw'].apply(convert_to_ml)

In [66]:
purchases[['size_raw', 'unit_size_ml']].head(100)

,size_raw,unit_size_ml
0,750ML,750.0
1,1.75L,1750.0
2,1.75L,1750.0
3,1.75L,1750.0
4,750ML,750.0
...,...,...
95,1.5L,1500.0
96,1.75L,1750.0
97,LITER,1000.0
98,750ML,750.0


In [67]:
mask_frac = purchases['size_raw'].str.contains(r'^\d+/\d+ML', regex=True, na=False)

In [68]:
fraction_parts = purchases.loc[mask_frac, 'size_raw'].str.extract(r'(\d+)/(\d+)ML').astype(float)
purchases.loc[mask_frac, 'pack_qty'] = fraction_parts[0]      
purchases.loc[mask_frac, 'unit_size_ml'] = fraction_parts[1]  

In [69]:
purchases.loc[mask_frac, ['size_raw', 'unit_size_ml', 'pack_qty']]

,size_raw,unit_size_ml,pack_qty
143327,3/100ML,100.0,3.0
143448,3/100ML,100.0,3.0
143515,3/100ML,100.0,3.0
185244,3/100ML,100.0,3.0
299775,3/100ML,100.0,3.0
...,...,...,...
2200870,3/100ML,100.0,3.0
2201250,3/100ML,100.0,3.0
2254224,3/100ML,100.0,3.0
2254582,3/100ML,100.0,3.0


In [70]:
mask_frac_oz = purchases['size_raw'].str.contains(r'^\d+/\d+(\.\d+)?\s*OZ$',regex=True,na=False)

C:\Users\UNBOX\AppData\Local\Temp\ipykernel_12608\122127811.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_frac_oz = purchases['size_raw'].str.contains(r'^\d+/\d+(\.\d+)?\s*OZ$',regex=True,na=False)


In [71]:
oz_parts = (
    purchases.loc[mask_frac_oz, 'size_raw']
    .str.extract(r'(\d+)/(\d+(\.\d+)?)\s*OZ')
)

purchases.loc[mask_frac_oz, 'pack_qty'] = oz_parts[0].astype(float)
purchases.loc[mask_frac_oz, 'unit_size_ml'] = oz_parts[1].astype(float) * 29.5735

In [72]:
purchases.loc[mask_frac_oz, ['size_raw', 'unit_size_ml', 'pack_qty']]

,size_raw,unit_size_ml,pack_qty
631123,5/2OZ,59.147,5.0
631124,5/2OZ,59.147,5.0
631125,5/2OZ,59.147,5.0
631126,5/2OZ,59.147,5.0
631127,5/2OZ,59.147,5.0
631128,5/2OZ,59.147,5.0
631129,5/2OZ,59.147,5.0
631130,5/2OZ,59.147,5.0
631131,5/2OZ,59.147,5.0
631132,5/2OZ,59.147,5.0


In [73]:
purchases['unit_size_ml'].unique()

array([7.5000000e+02, 1.7500000e+03, 2.0000000e+02, 1.5000000e+03,
       5.0000000e+01, 1.0000000e+03, 3.7500000e+02, 1.0000000e+02,
       3.0000000e+03, 1.8700000e+02, 2.5000000e+02, 5.0000000e+02,
       5.0000000e+03, 4.0000000e+03, 1.8000000e+02, 1.4786750e+02,
       1.8000000e+04, 3.0000000e+02, 7.2000000e+02, 3.3000000e+02,
       6.5061700e+02, 5.9147000e+01, 6.0000000e+03, 2.0000000e+04,
       4.0000000e+02,           nan, 7.3933750e+02, 1.9500000e+04,
       9.0000000e+03, 4.8453248e+05])

In [74]:
# Calculate Total Size
purchases['Size_ML'] = purchases['unit_size_ml'] * purchases['pack_qty']

In [75]:
# Validate Results
purchases[['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']].head(100)

,Size,unit_size_ml,pack_qty,Size_ML
0,750mL,750.0,1.0,750.0
1,1.75L,1750.0,1.0,1750.0
2,1.75L,1750.0,1.0,1750.0
3,1.75L,1750.0,1.0,1750.0
4,750mL,750.0,1.0,750.0
...,...,...,...,...
95,1.5L,1500.0,1.0,1500.0
96,1.75L,1750.0,1.0,1750.0
97,Liter,1000.0,1.0,1000.0
98,750mL,750.0,1.0,750.0


In [76]:
purchases['Size_ML'].isnull().sum()

np.int64(3)

In [77]:
purchases['Size'].unique()

array(['750mL', '1.75L', '200mL', '1.5L', '50mL', 'Liter', '375mL',
       '100mL', '750mL 2 Pk', '3L', '187mL 4 Pk', '250mL 4 Pk', '500mL',
       '187mL', '100mL 4 Pk', '5L', '4L', '375mL 2 Pk', '187mL 3 Pk',
       '250mL', '180mL', '50mL 5 Pk', '200mL 4 Pk', '5.0 Oz', '18L',
       '300mL', '720mL', '50mL 3 Pk', '50mL 4 Pk', '330mL', '750mL 3 Pk',
       '3/100mL', '750mL + 2/', '750mL 4 Pk', '200mL 3 Pk', '375mL 3 Pk',
       '50mL 12 Pk', '22.0 Oz', '750mL + 3/', '200mL 5 Pk', '750mL + 1/',
       '5/2 oz', '6L', '20L', '400mL', 'Unknown', '25.0 Oz', '187mL 2 Pk',
       '19.5L', '750mL 6 Pk', '9L', '128.0 Gal'], dtype=object)

In [78]:
# List of specific values
target_sizes = [
    '750mL', 
    '1.75L',
    'Liter',
    '750mL 2 Pk', 
    '3L',
    '5.0 Oz', 
    '3/100mL',
    '750mL + 3/', 
    '5/2 oz',
    'Unknown',
    '25.0 Oz',
    '128.0 Gal'
]

# Filter and display
result = purchases[purchases['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

               Size  unit_size_ml  pack_qty      Size_ML
0             750mL      750.0000       1.0     750.0000
2303767       1.75L     1750.0000       1.0    1750.0000
933462        Liter     1000.0000       1.0    1000.0000
104680   750mL 2 Pk      750.0000       2.0    1500.0000
2113114          3L     3000.0000       1.0    3000.0000
305855       5.0 Oz      147.8675       1.0     147.8675
882793      3/100mL      100.0000       3.0     300.0000
1908448  750mL + 3/      750.0000       4.0    3000.0000
631131       5/2 oz       59.1470       5.0     295.7350
1116302     Unknown           NaN       1.0          NaN
1242618     25.0 Oz      739.3375       1.0     739.3375
2103342   128.0 Gal   484532.4800       1.0  484532.4800


In [79]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 20 columns):
 #   Column          Dtype         
---  ------          -----         
 0   InventoryId     object        
 1   Store           int64         
 2   Brand           int64         
 3   Description     object        
 4   Size            object        
 5   VendorNumber    int64         
 6   VendorName      object        
 7   PONumber        int64         
 8   PODate          datetime64[ns]
 9   ReceivingDate   datetime64[ns]
 10  InvoiceDate     datetime64[ns]
 11  PayDate         datetime64[ns]
 12  PurchasePrice   float64       
 13  Quantity        int64         
 14  Dollars         float64       
 15  Classification  int64         
 16  size_raw        object        
 17  pack_qty        float64       
 18  unit_size_ml    float64       
 19  Size_ML         float64       
dtypes: datetime64[ns](4), float64(5), int64(6), object(5)
memory usage: 362.0+ MB


In [80]:
#check for null values
pd.isnull(purchases).sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
VendorNumber      0
VendorName        0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
size_raw          0
pack_qty          0
unit_size_ml      3
Size_ML           3
dtype: int64

In [81]:
purchases.drop(['Size','size_raw', 'unit_size_ml', 'pack_qty'], axis=1, inplace=True)

In [82]:
# Drop the null Values
purchases.dropna(subset=['Size_ML'], inplace=True)

In [83]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2372471 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype         
---  ------          -----         
 0   InventoryId     object        
 1   Store           int64         
 2   Brand           int64         
 3   Description     object        
 4   VendorNumber    int64         
 5   VendorName      object        
 6   PONumber        int64         
 7   PODate          datetime64[ns]
 8   ReceivingDate   datetime64[ns]
 9   InvoiceDate     datetime64[ns]
 10  PayDate         datetime64[ns]
 11  PurchasePrice   float64       
 12  Quantity        int64         
 13  Dollars         float64       
 14  Classification  int64         
 15  Size_ML         float64       
dtypes: datetime64[ns](4), float64(3), int64(6), object(3)
memory usage: 307.7+ MB


In [84]:
#check for null values
pd.isnull(purchases).sum()

InventoryId       0
Store             0
Brand             0
Description       0
VendorNumber      0
VendorName        0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
Size_ML           0
dtype: int64

In [85]:
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification',
       'Size_ML'],
      dtype='object')

In [86]:
# 1. Define the new column order
new_order = [
    'InventoryId', 'Store', 'Brand', 'Description','Size_ML',
    'VendorNumber','VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
    'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'
]

# 2. Reorder the dataframe
purchases = purchases[new_order]

# 3. Verify the change
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size_ML',
       'VendorNumber', 'VendorName', 'PONumber', 'PODate', 'ReceivingDate',
       'InvoiceDate', 'PayDate', 'PurchasePrice', 'Quantity', 'Dollars',
       'Classification'],
      dtype='object')

In [87]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2372471 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype         
---  ------          -----         
 0   InventoryId     object        
 1   Store           int64         
 2   Brand           int64         
 3   Description     object        
 4   Size_ML         float64       
 5   VendorNumber    int64         
 6   VendorName      object        
 7   PONumber        int64         
 8   PODate          datetime64[ns]
 9   ReceivingDate   datetime64[ns]
 10  InvoiceDate     datetime64[ns]
 11  PayDate         datetime64[ns]
 12  PurchasePrice   float64       
 13  Quantity        int64         
 14  Dollars         float64       
 15  Classification  int64         
dtypes: datetime64[ns](4), float64(3), int64(6), object(3)
memory usage: 307.7+ MB


In [88]:
purchases.shape

(2372471, 16)

In [89]:
# Approx size in MB
purchases.memory_usage(deep=True).sum() / (1024**2)

np.float64(717.7307472229004)

### Save Cleaned Data

In [90]:
purchases.to_csv(CLEAN_PATH + "Final_Purchase.csv", index=False)

### Step 5: Initial Inspection & Clean `InvoicePurchases12312016.csv`

In [91]:
invoice.head()

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2016-01-04,8124,2015-12-21,2016-02-16,6,214.26,3.47,NaN
1,4466,AMERICAN VINTAGE BEVERAGE,2016-01-07,8137,2015-12-22,2016-02-21,15,140.55,8.57,NaN
2,388,ATLANTIC IMPORTING COMPANY,2016-01-09,8169,2015-12-24,2016-02-16,5,106.60,4.61,NaN
3,480,BACARDI USA INC,2016-01-12,8106,2015-12-20,2016-02-05,10100,137483.78,2935.20,NaN
4,516,BANFI PRODUCTS CORP,2016-01-07,8170,2015-12-24,2016-02-12,1935,15527.25,429.20,NaN


In [92]:
print(f"No. of Rows: {invoice.shape[0]} \nNo. of Columns: {invoice.shape[1]}")

No. of Rows: 5543 
No. of Columns: 10


In [93]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VendorNumber  5543 non-null   int64  
 1   VendorName    5543 non-null   object 
 2   InvoiceDate   5543 non-null   object 
 3   PONumber      5543 non-null   int64  
 4   PODate        5543 non-null   object 
 5   PayDate       5543 non-null   object 
 6   Quantity      5543 non-null   int64  
 7   Dollars       5543 non-null   float64
 8   Freight       5543 non-null   float64
 9   Approval      374 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 433.2+ KB


In [94]:
#check for null values
pd.isnull(invoice).sum()

VendorNumber       0
VendorName         0
InvoiceDate        0
PONumber           0
PODate             0
PayDate            0
Quantity           0
Dollars            0
Freight            0
Approval        5169
dtype: int64

In [95]:
#check for duplicated values
invoice.duplicated().sum()

np.int64(0)

In [96]:
# Convert Date Columns
date_cols = ['InvoiceDate', 'PODate', 'PayDate']
for col in date_cols:
    invoice[col] = pd.to_datetime(invoice[col])

In [97]:
# Trim Text Columns
invoice['VendorName'] = invoice['VendorName'].str.strip()

In [98]:
# Fill nulls with a specific string
invoice['Approval']= invoice['Approval'].fillna('None')

In [99]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   VendorNumber  5543 non-null   int64         
 1   VendorName    5543 non-null   object        
 2   InvoiceDate   5543 non-null   datetime64[ns]
 3   PONumber      5543 non-null   int64         
 4   PODate        5543 non-null   datetime64[ns]
 5   PayDate       5543 non-null   datetime64[ns]
 6   Quantity      5543 non-null   int64         
 7   Dollars       5543 non-null   float64       
 8   Freight       5543 non-null   float64       
 9   Approval      5543 non-null   object        
dtypes: datetime64[ns](3), float64(2), int64(3), object(2)
memory usage: 433.2+ KB


In [100]:
#check for null values
pd.isnull(invoice).sum()

VendorNumber    0
VendorName      0
InvoiceDate     0
PONumber        0
PODate          0
PayDate         0
Quantity        0
Dollars         0
Freight         0
Approval        0
dtype: int64

In [101]:
# PO → Invoice Lead Time
invoice['PO_to_Invoice_Days'] = (
    invoice['InvoiceDate'] - invoice['PODate']
).dt.days

In [102]:
# Invoice → Payment Cycle
invoice['Invoice_to_Pay_Days'] = (
    invoice['PayDate'] - invoice['InvoiceDate']
).dt.days

In [103]:
# Total Invoice Cost (Including Freight)
invoice['Total_Invoice_Cost'] = invoice['Dollars'] + invoice['Freight']

In [104]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   VendorNumber         5543 non-null   int64         
 1   VendorName           5543 non-null   object        
 2   InvoiceDate          5543 non-null   datetime64[ns]
 3   PONumber             5543 non-null   int64         
 4   PODate               5543 non-null   datetime64[ns]
 5   PayDate              5543 non-null   datetime64[ns]
 6   Quantity             5543 non-null   int64         
 7   Dollars              5543 non-null   float64       
 8   Freight              5543 non-null   float64       
 9   Approval             5543 non-null   object        
 10  PO_to_Invoice_Days   5543 non-null   int64         
 11  Invoice_to_Pay_Days  5543 non-null   int64         
 12  Total_Invoice_Cost   5543 non-null   float64       
dtypes: datetime64[ns](3), float64(3),

### Save Cleaned Data

In [105]:
invoice.to_csv(CLEAN_PATH + "Invoice.csv", index=False)

### Step 6: Initial Inspection & Clean `2017PurchasePricesDec.csv`

In [106]:
price.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [107]:
print(f"No. of Rows: {price.shape[0]} \nNo. of Columns: {price.shape[1]}")

No. of Rows: 12261 
No. of Columns: 9


In [108]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12261 non-null  int64  
 1   Description     12260 non-null  object 
 2   Price           12261 non-null  float64
 3   Size            12260 non-null  object 
 4   Volume          12260 non-null  object 
 5   Classification  12261 non-null  int64  
 6   PurchasePrice   12261 non-null  float64
 7   VendorNumber    12261 non-null  int64  
 8   VendorName      12261 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 862.2+ KB


In [109]:
#check for null values
pd.isnull(price).sum()

Brand             0
Description       1
Price             0
Size              1
Volume            1
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
dtype: int64

In [110]:
#check for duplicated values
price.duplicated().sum()

np.int64(0)

In [111]:
# Trim Text Columns
price['VendorName'] = price['VendorName'].str.strip()
price['Description'] = price['Description'].str.strip()

In [112]:
price[price['Description'].isnull()]

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
7915,4202,NaN,0.0,NaN,NaN,1,11.19,480,BACARDI USA INC


In [113]:
# Drop the null Values
price.dropna(subset=['Description'], inplace=True)

In [114]:
price['Gross_Margin'] = price['Price'] - price['PurchasePrice']

In [115]:
price['Margin_Percent'] = (
    price['Gross_Margin'] / price['Price'] * 100
).round(2)

In [116]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12260 entries, 0 to 12260
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12260 non-null  int64  
 1   Description     12260 non-null  object 
 2   Price           12260 non-null  float64
 3   Size            12260 non-null  object 
 4   Volume          12260 non-null  object 
 5   Classification  12260 non-null  int64  
 6   PurchasePrice   12260 non-null  float64
 7   VendorNumber    12260 non-null  int64  
 8   VendorName      12260 non-null  object 
 9   Gross_Margin    12260 non-null  float64
 10  Margin_Percent  12259 non-null  float64
dtypes: float64(4), int64(3), object(4)
memory usage: 1.1+ MB


In [117]:
#check for null values
pd.isnull(price).sum()

Brand             0
Description       0
Price             0
Size              0
Volume            0
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
Gross_Margin      0
Margin_Percent    1
dtype: int64

In [118]:
price[price['Margin_Percent'].isnull()]

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Gross_Margin,Margin_Percent
10803,2166,The Macallan Double Cask 12,0.0,750mL,750,1,0.0,2561,EDRINGTON AMERICAS,0.0,NaN


In [119]:
# Fill nulls with a specific string
price['Margin_Percent'] = price['Margin_Percent'].fillna(0)

In [120]:
price[price['Margin_Percent'].isnull()]

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Gross_Margin,Margin_Percent


In [121]:
#check for null values
pd.isnull(price).sum()

Brand             0
Description       0
Price             0
Size              0
Volume            0
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
Gross_Margin      0
Margin_Percent    0
dtype: int64

In [122]:
price['Size'].unique()

array(['750mL', '1000mL', '1750mL', '50mL', '375mL', '100mL 4 Pk',
       '50mL 5 Pk', '100mL', '200mL', '50mL 4 Pk', '50mL 3 Pk', '300mL',
       '200mL 4 Pk', 'Unknown', '750mL 2 Pk', '250mL 4 Pk', '1500mL',
       '3000mL', '5000mL', '4000mL', '187mL 4 Pk', '150mL', '187mL',
       '500mL', '720mL', '650mL', '200mL 3 Pk', '187mL 3 Pk', '330mL',
       '250mL', '750mL + 3/', '18000mL', '180mL', '750mL + 4/', '6000mL',
       '1000mL 2 Pk', '750mL 12 P', '750mL 6 Pk', '20mL 5 Pk',
       '375mL 2 Pk', '20000mL', '50mL 12 Pk', '750mL 3 Pk', '375mL 3 Pk',
       '750mL + 1/', '200mL 5 Pk', '162.5mL', '400mL', '1100ml', '600mL',
       '19500mL', '560mL', '3750mL', '750mL  3', '1500ml', '9000mL'],
      dtype=object)

In [123]:
price['Volume'].unique()

array(['750', '1000', '1750', '50', '375', '100', '200', '300', 'Unknown',
       '250', '1500', '3000', '5000', '4000', '187', '150', '500', '720',
       '650', '330', '18000', '180', '6000', '20', '20000', '162.5',
       '400', '1100', '600', '19500', '560', '3750', '9000'], dtype=object)

In [124]:
# Create a Copy Column
price['size_raw'] = price['Size'].copy()

In [125]:
# Normalize Text
price['size_raw'] = (
    price['size_raw']
    .str.strip()
    .str.replace(' ', '')
    .str.upper()
)

In [126]:
price['size_raw'].unique()

array(['750ML', '1000ML', '1750ML', '50ML', '375ML', '100ML4PK',
       '50ML5PK', '100ML', '200ML', '50ML4PK', '50ML3PK', '300ML',
       '200ML4PK', 'UNKNOWN', '750ML2PK', '250ML4PK', '1500ML', '3000ML',
       '5000ML', '4000ML', '187ML4PK', '150ML', '187ML', '500ML', '720ML',
       '650ML', '200ML3PK', '187ML3PK', '330ML', '250ML', '750ML+3/',
       '18000ML', '180ML', '750ML+4/', '6000ML', '1000ML2PK', '750ML12P',
       '750ML6PK', '20ML5PK', '375ML2PK', '20000ML', '50ML12PK',
       '750ML3PK', '375ML3PK', '750ML+1/', '200ML5PK', '162.5ML', '400ML',
       '1100ML', '600ML', '19500ML', '560ML', '3750ML', '750ML3',
       '9000ML'], dtype=object)

In [127]:
# Default values
price['pack_qty'] = 1
price['unit_size_ml'] = np.nan

In [128]:
# Extract base ML
price['unit_size_ml'] = (
    price['size_raw']
    .str.extract(r'(\d+\.?\d*)ML')[0]
    .astype(float)
)

In [129]:
# Handle standard packs (4PK, 6PK, 12PK)
mask_pk = price['size_raw'].str.contains(r'PK', na=False)

price.loc[mask_pk, 'pack_qty'] = (
    price.loc[mask_pk, 'size_raw']
    .str.extract(r'(\d+)PK')[0]
    .astype(float)
)

In [130]:
# Handle bonus bottles (+3/, +4/)
mask_bonus = price['size_raw'].str.contains(r'\+\d+/', na=False)

bonus = (
    price.loc[mask_bonus, 'size_raw']
    .str.extract(r'\+(\d+)/')[0]
    .astype(float)
)

price.loc[mask_bonus, 'pack_qty'] = price.loc[mask_bonus, 'pack_qty'] + bonus

In [131]:
# Fix malformed rows (750ML3, 750ML12P)
mask_bad = price['size_raw'].str.match(r'\d+ML\d+$')

price.loc[mask_bad, 'pack_qty'] = (
    price.loc[mask_bad, 'size_raw']
    .str.extract(r'ML(\d+)')[0]
    .astype(float)
)

In [132]:
# Handle Unknown
price.loc[price['size_raw'] == 'UNKNOWN', ['unit_size_ml', 'pack_qty']] = np.nan

In [133]:
# Calculate Total Size
price['total_size_ml'] = price['unit_size_ml'] * price['pack_qty']

In [134]:
# Validate Results
price[['Size', 'unit_size_ml', 'pack_qty', 'total_size_ml']].head(100)

,Size,unit_size_ml,pack_qty,total_size_ml
0,750mL,750.0,1.0,750.0
1,750mL,750.0,1.0,750.0
2,750mL,750.0,1.0,750.0
3,750mL,750.0,1.0,750.0
4,750mL,750.0,1.0,750.0
...,...,...,...,...
95,1000mL,1000.0,1.0,1000.0
96,1000mL,1000.0,1.0,1000.0
97,750mL,750.0,1.0,750.0
98,750mL,750.0,1.0,750.0


In [135]:
# List of specific values
target_sizes = [
    '750mL', '100mL 4 Pk','Unknown','750mL + 3/', '18000mL','750mL + 4/',
     '750mL 12 P','750mL + 1/','162.5mL','750mL  3'
]

# Filter and display
result = price[price['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'total_size_ml','Volume']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

             Size  unit_size_ml  pack_qty  total_size_ml   Volume
0           750mL         750.0       1.0          750.0      750
113    100mL 4 Pk         100.0       4.0          400.0      100
5921      Unknown           NaN       NaN            NaN  Unknown
3974   750mL + 3/         750.0       4.0         3000.0      750
8564      18000mL       18000.0       1.0        18000.0    18000
5432   750mL + 4/         750.0       5.0         3750.0      750
7677   750mL 12 P         750.0       1.0          750.0      750
10191  750mL + 1/         750.0       2.0         1500.0      750
10267     162.5mL         162.5       1.0          162.5    162.5
11803    750mL  3         750.0       3.0         2250.0      750


In [136]:
# This overwrites the existing Volume column with the values from total_size_ml
price['Volume'] = price['total_size_ml']

In [137]:
# List of specific values
target_sizes = [
    '750mL', '100mL 4 Pk','Unknown','750mL + 3/', '18000mL','750mL + 4/',
     '750mL 12 P','750mL + 1/','162.5mL','750mL  3'
]

# Filter and display
result = price[price['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'total_size_ml','Volume']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

             Size  unit_size_ml  pack_qty  total_size_ml   Volume
0           750mL         750.0       1.0          750.0    750.0
113    100mL 4 Pk         100.0       4.0          400.0    400.0
5921      Unknown           NaN       NaN            NaN      NaN
3974   750mL + 3/         750.0       4.0         3000.0   3000.0
8564      18000mL       18000.0       1.0        18000.0  18000.0
5432   750mL + 4/         750.0       5.0         3750.0   3750.0
7677   750mL 12 P         750.0       1.0          750.0    750.0
10191  750mL + 1/         750.0       2.0         1500.0   1500.0
10267     162.5mL         162.5       1.0          162.5    162.5
11803    750mL  3         750.0       3.0         2250.0   2250.0


In [138]:
price[price['Volume'].isnull()]

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Gross_Margin,Margin_Percent,size_raw,pack_qty,unit_size_ml,total_size_ml
542,2993,Angostura Bitters,7.49,Unknown,NaN,1,5.39,5895,"Mizkan Americas, Inc.",2.10,28.04,UNKNOWN,NaN,NaN,NaN
5921,9908,Tito's Copper Mug 2 Pack,21.01,Unknown,NaN,1,16.15,4425,MARTIGNETTI COMPANIES,4.86,23.13,UNKNOWN,NaN,NaN,NaN
8795,8992,Group 92,1.99,Unknown,NaN,1,1.43,1703,ALISA CARR BEVERAGES,0.56,28.14,UNKNOWN,NaN,NaN,NaN
10009,90590,Overture Champagne 2Glass Pk,19.95,Unknown,NaN,2,13.12,4425,MARTIGNETTI COMPANIES,6.83,34.24,UNKNOWN,NaN,NaN,NaN


In [139]:
price[price['Size'] == 'Unknown']

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Gross_Margin,Margin_Percent,size_raw,pack_qty,unit_size_ml,total_size_ml
542,2993,Angostura Bitters,7.49,Unknown,NaN,1,5.39,5895,"Mizkan Americas, Inc.",2.10,28.04,UNKNOWN,NaN,NaN,NaN
5921,9908,Tito's Copper Mug 2 Pack,21.01,Unknown,NaN,1,16.15,4425,MARTIGNETTI COMPANIES,4.86,23.13,UNKNOWN,NaN,NaN,NaN
8795,8992,Group 92,1.99,Unknown,NaN,1,1.43,1703,ALISA CARR BEVERAGES,0.56,28.14,UNKNOWN,NaN,NaN,NaN
10009,90590,Overture Champagne 2Glass Pk,19.95,Unknown,NaN,2,13.12,4425,MARTIGNETTI COMPANIES,6.83,34.24,UNKNOWN,NaN,NaN,NaN


In [140]:
# Fill nulls with a specific string
price['Volume'] = price['Volume'].fillna(0)

In [141]:
price.drop(['size_raw', 'unit_size_ml', 'pack_qty','total_size_ml'], axis=1, inplace=True)

In [142]:
#check for null values
pd.isnull(price).sum()

Brand             0
Description       0
Price             0
Size              0
Volume            0
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
Gross_Margin      0
Margin_Percent    0
dtype: int64

In [143]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12260 entries, 0 to 12260
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12260 non-null  int64  
 1   Description     12260 non-null  object 
 2   Price           12260 non-null  float64
 3   Size            12260 non-null  object 
 4   Volume          12260 non-null  float64
 5   Classification  12260 non-null  int64  
 6   PurchasePrice   12260 non-null  float64
 7   VendorNumber    12260 non-null  int64  
 8   VendorName      12260 non-null  object 
 9   Gross_Margin    12260 non-null  float64
 10  Margin_Percent  12260 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 1.1+ MB


In [144]:
price.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,Gross_Margin,Margin_Percent
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750.0,1,9.28,8320,SHAW ROSS INT L IMP LTD,3.71,28.56
1,62,Herradura Silver Tequila,36.99,750mL,750.0,1,28.67,1128,BROWN-FORMAN CORP,8.32,22.49
2,63,Herradura Reposado Tequila,38.99,750mL,750.0,1,30.46,1128,BROWN-FORMAN CORP,8.53,21.88
3,72,No. 3 London Dry Gin,34.99,750mL,750.0,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP,8.88,25.38
4,75,Three Olives Tomato Vodka,14.99,750mL,750.0,1,10.94,7245,PROXIMO SPIRITS INC.,4.05,27.02


### Save Cleaned Data

In [145]:
price.to_csv(CLEAN_PATH + "Purchase_Price.csv", index=False)

### Step 7: Initial Inspection & Clean `BegInvFINAL12312016.csv`

In [146]:
beg_inv.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2016-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2016-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2016-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2016-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2016-01-01


In [147]:
print(f"No. of Rows: {beg_inv.shape[0]} \nNo. of Columns: {beg_inv.shape[1]}")

No. of Rows: 206529 
No. of Columns: 9


In [148]:
beg_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  206529 non-null  object 
 1   Store        206529 non-null  int64  
 2   City         206529 non-null  object 
 3   Brand        206529 non-null  int64  
 4   Description  206529 non-null  object 
 5   Size         206529 non-null  object 
 6   onHand       206529 non-null  int64  
 7   Price        206529 non-null  float64
 8   startDate    206529 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 14.2+ MB


In [149]:
#check for null values
pd.isnull(beg_inv).sum()

InventoryId    0
Store          0
City           0
Brand          0
Description    0
Size           0
onHand         0
Price          0
startDate      0
dtype: int64

In [150]:
#check for duplicated values
beg_inv.duplicated().sum()

np.int64(0)

In [151]:
# Convert Date Column
beg_inv['startDate'] = pd.to_datetime(beg_inv['startDate'])

In [152]:
# Trim Text Columns
beg_inv['InventoryId'] = beg_inv['InventoryId'].str.strip()
beg_inv['City'] = beg_inv['City'].str.strip()
beg_inv['Description'] = beg_inv['Description'].str.strip()

In [153]:
beg_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  206529 non-null  object        
 1   Store        206529 non-null  int64         
 2   City         206529 non-null  object        
 3   Brand        206529 non-null  int64         
 4   Description  206529 non-null  object        
 5   Size         206529 non-null  object        
 6   onHand       206529 non-null  int64         
 7   Price        206529 non-null  float64       
 8   startDate    206529 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 14.2+ MB


In [154]:
beg_inv['Size'].unique()

array(['750mL', 'Liter', '1.75L', '50mL', '375mL', '50mL 4 Pk',
       '375mL 2 Pk', '750mL 3 Pk', '375mL 3 Pk', '750mL + 3/',
       '200mL 3 Pk', '50mL 3 Pk', '100mL 4 Pk', '100mL', '50mL 5 Pk',
       '200mL 4 Pk', '250mL 4 Pk', '750mL 2 Pk', '200mL', '3/100mL',
       '1.5L', '5L', '4L', '187mL 4 Pk', '3L', '5.0 Oz', '187mL', '500mL',
       '187mL 3 Pk', '300mL', '50mL 12 Pk', '330mL', '250mL', '18L',
       '750mL + 2/', '180mL', '720mL', '22.0 Oz', '6L', '20L',
       '750mL + 4/'], dtype=object)

In [155]:
# Create a Copy Column
beg_inv['size_raw'] = beg_inv['Size'].copy()

In [156]:
# Normalize Text
beg_inv['size_raw'] = (
    beg_inv['size_raw']
    .str.strip()
    .str.replace(' ', '')
    .str.upper()
)

In [157]:
beg_inv['size_raw'].unique()

array(['750ML', 'LITER', '1.75L', '50ML', '375ML', '50ML4PK', '375ML2PK',
       '750ML3PK', '375ML3PK', '750ML+3/', '200ML3PK', '50ML3PK',
       '100ML4PK', '100ML', '50ML5PK', '200ML4PK', '250ML4PK', '750ML2PK',
       '200ML', '3/100ML', '1.5L', '5L', '4L', '187ML4PK', '3L', '5.0OZ',
       '187ML', '500ML', '187ML3PK', '300ML', '50ML12PK', '330ML',
       '250ML', '18L', '750ML+2/', '180ML', '720ML', '22.0OZ', '6L',
       '20L', '750ML+4/'], dtype=object)

In [158]:
# Extract Pack Quantity
beg_inv['pack_qty'] = (
    beg_inv['size_raw']
    .str.extract(r'(\d+)PK')
    .astype(float)
)

beg_inv['pack_qty'] = beg_inv['pack_qty'].fillna(1)

In [159]:
# Handle Bonus Bottles (+ 2/, + 3/)
# Example:- 750ML+3/ → 4 bottles total

bonus = beg_inv['size_raw'].str.extract(r'\+(\d+)/')[0].astype(float).fillna(0)

In [160]:
beg_inv['pack_qty'] = beg_inv['pack_qty'] + bonus

In [161]:
beg_inv[['Size', 'pack_qty']].head(100)

,Size,pack_qty
0,750mL,1.0
1,750mL,1.0
2,750mL,1.0
3,750mL,1.0
4,750mL,1.0
...,...,...
95,750mL,1.0
96,Liter,1.0
97,Liter,1.0
98,Liter,1.0


In [162]:
# Extract Base Size (mL, L, Oz) & Convert everything into mL

def convert_to_ml(size):
    # Handle explicit Liter
    if size == 'LITER':
        return 1000.0
    
    num = re.search(r'\d+\.?\d*', size)
    
    if num:
        value = float(num.group())
        if 'ML' in size:
            return value
        elif 'L' in size:
            return value * 1000
        elif 'OZ' in size:
            return value * 29.5735
            
    return np.nan

In [163]:
# Apply Size Conversion
beg_inv['unit_size_ml'] = beg_inv['size_raw'].apply(convert_to_ml)

In [164]:
beg_inv[['size_raw', 'unit_size_ml']].head(100)

,size_raw,unit_size_ml
0,750ML,750.0
1,750ML,750.0
2,750ML,750.0
3,750ML,750.0
4,750ML,750.0
...,...,...
95,750ML,750.0
96,LITER,1000.0
97,LITER,1000.0
98,LITER,1000.0


In [165]:
mask_frac = beg_inv['size_raw'].str.contains(r'^\d+/\d+ML', regex=True, na=False)

In [166]:
fraction_parts = beg_inv.loc[mask_frac, 'size_raw'].str.extract(r'(\d+)/(\d+)ML').astype(float)
beg_inv.loc[mask_frac, 'pack_qty'] = fraction_parts[0]      
beg_inv.loc[mask_frac, 'unit_size_ml'] = fraction_parts[1]  

In [167]:
# Calculate Total Size
beg_inv['Size_ML'] = beg_inv['unit_size_ml'] * beg_inv['pack_qty']

In [168]:
# Validate Results
beg_inv[['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']].head(100)

,Size,unit_size_ml,pack_qty,Size_ML
0,750mL,750.0,1.0,750.0
1,750mL,750.0,1.0,750.0
2,750mL,750.0,1.0,750.0
3,750mL,750.0,1.0,750.0
4,750mL,750.0,1.0,750.0
...,...,...,...,...
95,750mL,750.0,1.0,750.0
96,Liter,1000.0,1.0,1000.0
97,Liter,1000.0,1.0,1000.0
98,Liter,1000.0,1.0,1000.0


In [169]:
# List of specific values
target_sizes = [
    '750mL', 
    '375mL 2 Pk', 
    '750mL + 3/', 
    'Liter', 
    '1.75L', 
    '5.0 Oz', 
    '3/100mL', 
    '18L'
]

# Filter and display
result = beg_inv[beg_inv['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

              Size  unit_size_ml  pack_qty     Size_ML
0            750mL      750.0000       1.0    750.0000
78508   375mL 2 Pk      375.0000       2.0    750.0000
46818   750mL + 3/      750.0000       4.0   3000.0000
135174       Liter     1000.0000       1.0   1000.0000
87991        1.75L     1750.0000       1.0   1750.0000
205008      5.0 Oz      147.8675       1.0    147.8675
142141     3/100mL      100.0000       3.0    300.0000
109962         18L    18000.0000       1.0  18000.0000


In [170]:
beg_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InventoryId   206529 non-null  object        
 1   Store         206529 non-null  int64         
 2   City          206529 non-null  object        
 3   Brand         206529 non-null  int64         
 4   Description   206529 non-null  object        
 5   Size          206529 non-null  object        
 6   onHand        206529 non-null  int64         
 7   Price         206529 non-null  float64       
 8   startDate     206529 non-null  datetime64[ns]
 9   size_raw      206529 non-null  object        
 10  pack_qty      206529 non-null  float64       
 11  unit_size_ml  206529 non-null  float64       
 12  Size_ML       206529 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 20.5+ MB


In [171]:
beg_inv.drop(['Size','size_raw', 'unit_size_ml', 'pack_qty'], axis=1, inplace=True)

In [172]:
beg_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  206529 non-null  object        
 1   Store        206529 non-null  int64         
 2   City         206529 non-null  object        
 3   Brand        206529 non-null  int64         
 4   Description  206529 non-null  object        
 5   onHand       206529 non-null  int64         
 6   Price        206529 non-null  float64       
 7   startDate    206529 non-null  datetime64[ns]
 8   Size_ML      206529 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 14.2+ MB


In [173]:
beg_inv.columns

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'onHand',
       'Price', 'startDate', 'Size_ML'],
      dtype='object')

In [174]:
# 1. Define the new column order
new_order = [
    'InventoryId', 'Store', 'City', 'Brand', 'Description','Size_ML', 'onHand',
       'Price', 'startDate'
]

# 2. Reorder the dataframe
beg_inv = beg_inv[new_order]

# 3. Verify the change
beg_inv.columns

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size_ML',
       'onHand', 'Price', 'startDate'],
      dtype='object')

In [175]:
beg_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  206529 non-null  object        
 1   Store        206529 non-null  int64         
 2   City         206529 non-null  object        
 3   Brand        206529 non-null  int64         
 4   Description  206529 non-null  object        
 5   Size_ML      206529 non-null  float64       
 6   onHand       206529 non-null  int64         
 7   Price        206529 non-null  float64       
 8   startDate    206529 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 14.2+ MB


In [176]:
beg_inv.shape

(206529, 9)

### Save Cleaned Data

In [177]:
beg_inv.to_csv(CLEAN_PATH + "Beg_Inv.csv", index=False)

### Step 8: Initial Inspection & Clean `EndInvFINAL12312016.csv`

In [178]:
end_inv.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2016-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2016-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2016-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2016-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2016-12-31


In [179]:
print(f"No. of Rows: {end_inv.shape[0]} \nNo. of Columns: {end_inv.shape[1]}")

No. of Rows: 224489 
No. of Columns: 9


In [180]:
end_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  224489 non-null  object 
 1   Store        224489 non-null  int64  
 2   City         223205 non-null  object 
 3   Brand        224489 non-null  int64  
 4   Description  224489 non-null  object 
 5   Size         224489 non-null  object 
 6   onHand       224489 non-null  int64  
 7   Price        224489 non-null  float64
 8   endDate      224489 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4+ MB


In [181]:
#check for null values
pd.isnull(end_inv).sum()

InventoryId       0
Store             0
City           1284
Brand             0
Description       0
Size              0
onHand            0
Price             0
endDate           0
dtype: int64

In [182]:
#check for duplicated values
end_inv.duplicated().sum()

np.int64(0)

In [183]:
# Convert Date Column
end_inv['endDate'] = pd.to_datetime(end_inv['endDate'])

In [184]:
# Trim Text Columns
end_inv['InventoryId'] = end_inv['InventoryId'].str.strip()
end_inv['City'] = end_inv['City'].str.strip()
end_inv['Description'] = end_inv['Description'].str.strip()

In [185]:
# Fill nulls with a specific string
end_inv['City'] = end_inv['City'].fillna('Unknown')

In [186]:
end_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  224489 non-null  object        
 1   Store        224489 non-null  int64         
 2   City         224489 non-null  object        
 3   Brand        224489 non-null  int64         
 4   Description  224489 non-null  object        
 5   Size         224489 non-null  object        
 6   onHand       224489 non-null  int64         
 7   Price        224489 non-null  float64       
 8   endDate      224489 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 15.4+ MB


In [187]:
#check for null values
pd.isnull(end_inv).sum()

InventoryId    0
Store          0
City           0
Brand          0
Description    0
Size           0
onHand         0
Price          0
endDate        0
dtype: int64

In [188]:
end_inv['Size'].unique()

array(['750mL', 'Liter', '750mL + 2/', '1.75L', '50mL', '375mL',
       '50mL 4 Pk', '200mL 3 Pk', '50mL 3 Pk', '750mL + 3/', '375mL 2 Pk',
       '100mL 4 Pk', '250mL', '100mL', '50mL 5 Pk', '3/100mL',
       '200mL 4 Pk', '250mL 4 Pk', '200mL', '1.5L', '5L', '4L',
       '187mL 4 Pk', '3L', '5.0 Oz', '187mL 3 Pk', '187mL', '500mL',
       '750mL 3 Pk', '400mL', '300mL', '330mL', '18L', '750mL 2 Pk',
       '5/2 oz', '720mL', '375mL 3 Pk', '750mL + 1/', '180mL',
       '200mL 5 Pk', '6L', '187mL 2 Pk', '750mL 4 Pk', '750mL 6 Pk',
       '750mL + 4/', '25.0', '128.0 Gal'], dtype=object)

In [189]:
end_inv[end_inv['Size']== '25.0']

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
181834,69_MOUNTMEND_169,69,MOUNTMEND,169,Twisted Shotz Multi Pack,25.0,6,16.99,2016-12-31


In [190]:
end_inv['Size'] = end_inv['Size'].astype(str).str.strip().replace('25.0', '25.0 Oz')

In [191]:
end_inv['Size'].unique()

array(['750mL', 'Liter', '750mL + 2/', '1.75L', '50mL', '375mL',
       '50mL 4 Pk', '200mL 3 Pk', '50mL 3 Pk', '750mL + 3/', '375mL 2 Pk',
       '100mL 4 Pk', '250mL', '100mL', '50mL 5 Pk', '3/100mL',
       '200mL 4 Pk', '250mL 4 Pk', '200mL', '1.5L', '5L', '4L',
       '187mL 4 Pk', '3L', '5.0 Oz', '187mL 3 Pk', '187mL', '500mL',
       '750mL 3 Pk', '400mL', '300mL', '330mL', '18L', '750mL 2 Pk',
       '5/2 oz', '720mL', '375mL 3 Pk', '750mL + 1/', '180mL',
       '200mL 5 Pk', '6L', '187mL 2 Pk', '750mL 4 Pk', '750mL 6 Pk',
       '750mL + 4/', '25.0 Oz', '128.0 Gal'], dtype=object)

In [192]:
# Create a Copy Column
end_inv['size_raw'] = end_inv['Size'].copy()

In [193]:
# Normalize Text
end_inv['size_raw'] = (
    end_inv['size_raw']
    .str.strip()
    .str.replace(' ', '')
    .str.upper()
)

In [194]:
end_inv['size_raw'].unique()

array(['750ML', 'LITER', '750ML+2/', '1.75L', '50ML', '375ML', '50ML4PK',
       '200ML3PK', '50ML3PK', '750ML+3/', '375ML2PK', '100ML4PK', '250ML',
       '100ML', '50ML5PK', '3/100ML', '200ML4PK', '250ML4PK', '200ML',
       '1.5L', '5L', '4L', '187ML4PK', '3L', '5.0OZ', '187ML3PK', '187ML',
       '500ML', '750ML3PK', '400ML', '300ML', '330ML', '18L', '750ML2PK',
       '5/2OZ', '720ML', '375ML3PK', '750ML+1/', '180ML', '200ML5PK',
       '6L', '187ML2PK', '750ML4PK', '750ML6PK', '750ML+4/', '25.0OZ',
       '128.0GAL'], dtype=object)

In [195]:
# Extract Pack Quantity
end_inv['pack_qty'] = (
    end_inv['size_raw']
    .str.extract(r'(\d+)PK')
    .astype(float)
)

end_inv['pack_qty'] = end_inv['pack_qty'].fillna(1)

In [196]:
# Handle Bonus Bottles (+ 2/, + 3/)
# Example:- 750ML+3/ → 4 bottles total

bonus = end_inv['size_raw'].str.extract(r'\+(\d+)/')[0].astype(float).fillna(0)

In [197]:
end_inv['pack_qty'] = end_inv['pack_qty'] + bonus

In [198]:
end_inv[['Size', 'pack_qty']].head(100)

,Size,pack_qty
0,750mL,1.0
1,750mL,1.0
2,750mL,1.0
3,750mL,1.0
4,750mL,1.0
...,...,...
95,Liter,1.0
96,750mL,1.0
97,Liter,1.0
98,Liter,1.0


In [199]:
# Extract Base Size (mL, L, Oz) & Convert everything into mL
def convert_to_ml(size):

    # # Handle Unknown
    # if size in ['UNKNOWN', '']:
    #     return np.nan

    # Gallons
    gal = re.match(r'(\d+\.?\d*)\s*GAL', size)
    if gal:
        return float(gal.group(1)) * 3785.41
    
    # Handle explicit Liter
    if size == 'LITER':
        return 1000.0

     # Ounces
    oz = re.match(r'(\d+\.?\d*)\s*OZ', size)
    if oz:
        return float(oz.group(1)) * 29.5735
    
    num = re.search(r'\d+\.?\d*', size)
    
    if num:
        value = float(num.group())
        if 'ML' in size:
            return value
        elif 'L' in size:
            return value * 1000
            
    return np.nan

In [200]:
# Apply Size Conversion
end_inv['unit_size_ml'] = end_inv['size_raw'].apply(convert_to_ml)

In [201]:
end_inv[['size_raw', 'unit_size_ml']].head(100)

,size_raw,unit_size_ml
0,750ML,750.0
1,750ML,750.0
2,750ML,750.0
3,750ML,750.0
4,750ML,750.0
...,...,...
95,LITER,1000.0
96,750ML,750.0
97,LITER,1000.0
98,LITER,1000.0


In [202]:
mask_frac = end_inv['size_raw'].str.contains(r'^\d+/\d+ML', regex=True, na=False)

In [203]:
fraction_parts = end_inv.loc[mask_frac, 'size_raw'].str.extract(r'(\d+)/(\d+)ML').astype(float)
end_inv.loc[mask_frac, 'pack_qty'] = fraction_parts[0]      
end_inv.loc[mask_frac, 'unit_size_ml'] = fraction_parts[1]  

In [204]:
end_inv.loc[mask_frac, ['size_raw', 'unit_size_ml', 'pack_qty']]

,size_raw,unit_size_ml,pack_qty
426,3/100ML,100.0,3.0
4447,3/100ML,100.0,3.0
5983,3/100ML,100.0,3.0
22263,3/100ML,100.0,3.0
31603,3/100ML,100.0,3.0
34327,3/100ML,100.0,3.0
53757,3/100ML,100.0,3.0
59535,3/100ML,100.0,3.0
69095,3/100ML,100.0,3.0
73206,3/100ML,100.0,3.0


In [205]:
mask_frac_oz = end_inv['size_raw'].str.contains(r'^\d+/\d+(\.\d+)?\s*OZ$',regex=True,na=False)

C:\Users\UNBOX\AppData\Local\Temp\ipykernel_12608\104905252.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_frac_oz = end_inv['size_raw'].str.contains(r'^\d+/\d+(\.\d+)?\s*OZ$',regex=True,na=False)


In [206]:
oz_parts = (
    end_inv.loc[mask_frac_oz, 'size_raw']
    .str.extract(r'(\d+)/(\d+(\.\d+)?)\s*OZ')
)

end_inv.loc[mask_frac_oz, 'pack_qty'] = oz_parts[0].astype(float)
end_inv.loc[mask_frac_oz, 'unit_size_ml'] = oz_parts[1].astype(float) * 29.5735

In [207]:
end_inv.loc[mask_frac_oz, ['size_raw', 'unit_size_ml', 'pack_qty']]

,size_raw,unit_size_ml,pack_qty
13846,5/2OZ,59.147,5.0
86160,5/2OZ,59.147,5.0
96149,5/2OZ,59.147,5.0
159685,5/2OZ,59.147,5.0
173104,5/2OZ,59.147,5.0
178245,5/2OZ,59.147,5.0
187544,5/2OZ,59.147,5.0
200240,5/2OZ,59.147,5.0
212720,5/2OZ,59.147,5.0
220898,5/2OZ,59.147,5.0


In [208]:
end_inv['unit_size_ml'].unique()

array([7.5000000e+02, 1.0000000e+03, 1.7500000e+03, 5.0000000e+01,
       3.7500000e+02, 2.0000000e+02, 1.0000000e+02, 2.5000000e+02,
       1.5000000e+03, 5.0000000e+03, 4.0000000e+03, 1.8700000e+02,
       3.0000000e+03, 1.4786750e+02, 5.0000000e+02, 4.0000000e+02,
       3.0000000e+02, 3.3000000e+02, 1.8000000e+04, 5.9147000e+01,
       7.2000000e+02, 1.8000000e+02, 6.0000000e+03, 7.3933750e+02,
       4.8453248e+05])

In [209]:
# Calculate Total Size
end_inv['Size_ML'] = end_inv['unit_size_ml'] * end_inv['pack_qty']

In [210]:
# Validate Results
end_inv[['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']].head(100)

,Size,unit_size_ml,pack_qty,Size_ML
0,750mL,750.0,1.0,750.0
1,750mL,750.0,1.0,750.0
2,750mL,750.0,1.0,750.0
3,750mL,750.0,1.0,750.0
4,750mL,750.0,1.0,750.0
...,...,...,...,...
95,Liter,1000.0,1.0,1000.0
96,750mL,750.0,1.0,750.0
97,Liter,1000.0,1.0,1000.0
98,Liter,1000.0,1.0,1000.0


In [211]:
end_inv['Size_ML'].isnull().sum()

np.int64(0)

In [212]:
end_inv['Size'].unique()

array(['750mL', 'Liter', '750mL + 2/', '1.75L', '50mL', '375mL',
       '50mL 4 Pk', '200mL 3 Pk', '50mL 3 Pk', '750mL + 3/', '375mL 2 Pk',
       '100mL 4 Pk', '250mL', '100mL', '50mL 5 Pk', '3/100mL',
       '200mL 4 Pk', '250mL 4 Pk', '200mL', '1.5L', '5L', '4L',
       '187mL 4 Pk', '3L', '5.0 Oz', '187mL 3 Pk', '187mL', '500mL',
       '750mL 3 Pk', '400mL', '300mL', '330mL', '18L', '750mL 2 Pk',
       '5/2 oz', '720mL', '375mL 3 Pk', '750mL + 1/', '180mL',
       '200mL 5 Pk', '6L', '187mL 2 Pk', '750mL 4 Pk', '750mL 6 Pk',
       '750mL + 4/', '25.0 Oz', '128.0 Gal'], dtype=object)

In [213]:
# List of specific values
target_sizes = [
    '750mL', 
    '1.75L',
    'Liter',
    '750mL 2 Pk', 
    '3L',
    '5.0 Oz', 
    '3/100mL',
    '750mL + 3/', 
    '5/2 oz',
    '25.0 Oz',
    '128.0 Gal'
]

# Filter and display
result = end_inv[end_inv['Size'].isin(target_sizes)][['Size', 'unit_size_ml', 'pack_qty', 'Size_ML']]

# To ensure they appear in the exact order of your list:
result['Size'] = pd.Categorical(result['Size'], categories=target_sizes, ordered=True)
print(result.sort_values('Size').drop_duplicates())

              Size  unit_size_ml  pack_qty      Size_ML
0            750mL      750.0000       1.0     750.0000
19567        1.75L     1750.0000       1.0    1750.0000
163538       Liter     1000.0000       1.0    1000.0000
122536  750mL 2 Pk      750.0000       2.0    1500.0000
82424           3L     3000.0000       1.0    3000.0000
77741       5.0 Oz      147.8675       1.0     147.8675
153282     3/100mL      100.0000       3.0     300.0000
159737  750mL + 3/      750.0000       4.0    3000.0000
178245      5/2 oz       59.1470       5.0     295.7350
181834     25.0 Oz      739.3375       1.0     739.3375
200440   128.0 Gal   484532.4800       1.0  484532.4800


In [214]:
end_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InventoryId   224489 non-null  object        
 1   Store         224489 non-null  int64         
 2   City          224489 non-null  object        
 3   Brand         224489 non-null  int64         
 4   Description   224489 non-null  object        
 5   Size          224489 non-null  object        
 6   onHand        224489 non-null  int64         
 7   Price         224489 non-null  float64       
 8   endDate       224489 non-null  datetime64[ns]
 9   size_raw      224489 non-null  object        
 10  pack_qty      224489 non-null  float64       
 11  unit_size_ml  224489 non-null  float64       
 12  Size_ML       224489 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 22.3+ MB


In [215]:
#check for null values
pd.isnull(end_inv).sum()

InventoryId     0
Store           0
City            0
Brand           0
Description     0
Size            0
onHand          0
Price           0
endDate         0
size_raw        0
pack_qty        0
unit_size_ml    0
Size_ML         0
dtype: int64

In [216]:
end_inv.drop(['Size','size_raw', 'unit_size_ml', 'pack_qty'], axis=1, inplace=True)

In [217]:
end_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  224489 non-null  object        
 1   Store        224489 non-null  int64         
 2   City         224489 non-null  object        
 3   Brand        224489 non-null  int64         
 4   Description  224489 non-null  object        
 5   onHand       224489 non-null  int64         
 6   Price        224489 non-null  float64       
 7   endDate      224489 non-null  datetime64[ns]
 8   Size_ML      224489 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 15.4+ MB


In [218]:
end_inv.columns

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'onHand',
       'Price', 'endDate', 'Size_ML'],
      dtype='object')

In [219]:
# 1. Define the new column order
new_order = [
    'InventoryId', 'Store', 'City', 'Brand', 'Description' , 'Size_ML', 'onHand',
       'Price', 'endDate'
]

# 2. Reorder the dataframe
end_inv = end_inv[new_order]

# 3. Verify the change
end_inv.columns

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size_ML',
       'onHand', 'Price', 'endDate'],
      dtype='object')

In [220]:
end_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  224489 non-null  object        
 1   Store        224489 non-null  int64         
 2   City         224489 non-null  object        
 3   Brand        224489 non-null  int64         
 4   Description  224489 non-null  object        
 5   Size_ML      224489 non-null  float64       
 6   onHand       224489 non-null  int64         
 7   Price        224489 non-null  float64       
 8   endDate      224489 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 15.4+ MB


In [221]:
end_inv.shape

(224489, 9)

### Save Cleaned Data

In [222]:
end_inv.to_csv(CLEAN_PATH + "End_Inv.csv", index=False)